In [35]:
import cv2
from ultralytics import YOLO
import torchvision.ops as ops
import colorsys
import datetime

In [36]:
checkpoint = YOLO('../ml_models/yolov8_50epochs_last.pt')
# INPUT = 0 # webcam
INPUT = "../data/20230307_143020.mp4"  # video

In [37]:

def float_to_color(f):
    """
    Convert a float from 0 to 1 to a BGR color from red to yellow to green.
    """
    hue = 0.33 * f
    r, g, b = colorsys.hsv_to_rgb(hue, 1, 1)
    r = int(r * 255)
    g = int(g * 255)
    b = int(b * 255)
    return (b, g, r)


In [38]:

def nms_analysis(image, model):
    threshold = 0
    result = model(image)
    boxes = result[0].boxes.xyxy.cuda()
    scores = result[0].boxes.conf.cuda()

    # Non-Maximum Suppression
    indices = ops.nms(boxes, scores, threshold)

    cv2.putText(image, f"Paddles: {len(indices)}", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, .75,
                (255, 255, 255), 2)

    if indices.numel() == 0:
        return image

    boxes = boxes[indices]
    scores = scores[indices]
    for box, score in zip(boxes, scores):
        if score >= threshold:
            box = [int(i) for i in box]
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), float_to_color(score), 2)
            cv2.putText(image, f"Score: {round(float(score), 2)}", (box[2], box[3]), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 2)

    return image



In [39]:
def analyse_video():
    cap = cv2.VideoCapture(INPUT)


    now = datetime.datetime.now()
    date_string = now.strftime('%Y-%m-%d-%H-%M-%S')

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec for MP4 file
    out = cv2.VideoWriter(f"output{date_string}.mp4", fourcc, 30.0, (960, 540))  # output file name, codec, fps, frame size

    while cap.isOpened():
        ret, frame = cap.read()
        # frame = cv2.resize(frame, (960, 540))
        img = nms_analysis(frame, checkpoint)

        if img is None or img.shape[0] == 0 or img.shape[1] == 0:
            break
        cv2.imshow('image', img)

        # out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

analyse_video()


0: 480x800 (no detections), 10.5ms
Speed: 1.0ms preprocess, 10.5ms inference, 2.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 3.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 11.8ms
Speed: 1.0ms preprocess, 11.8ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 11.0ms
Speed: 0.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 10.5ms
Speed: 1.0ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 12.0ms
Speed: 0.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 11.4ms
Speed: 1.0ms preprocess, 11.4ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 (no detections), 11.0ms
Speed: 1.0ms preprocess, 11.0ms inf